In [12]:
%matplotlib inline
from numpy import *
import numpy as np
import math
import re
import glob

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn import linear_model

## b) Preprocessing and c) Split training and testing set
Since these sentences are online reviews, they may contain significant amounts of noise and garbage.

* Lowercase all of the words
* Lemmatization of all the words
* Strip punctuation
* Strip the stop words

for each file, please use the first 400 in- stances for each label as the training set and the remaining 100 instances as testing set. In total, there are 2400 reviews for training and 600 reviews for testing.

In [13]:
entries = [] # words for each entity for all files
labels = [] # sentiment label for all files

# for Spliting training and testing set
test_X = []
test_Y = []
train_X = []
train_Y = []

for filename in glob.glob('./data/*.txt'):
    f = open(filename, 'r')
    positiveCnt = 0; negativeCnt = 0
    sampThreshold = 800
    for line in f:
        temp = line.split('\t') # split strings and sentiment label
        label = temp[1][0]
        labels.append(label) # add sentiment label

        # remove all signs and spaces
        # 'filter' is for removing all null strings after regluar expression applied
        # 're' is for applying regluar exp so that signs would be remove
        # 'lower' is for make all words become lower case
        sentence = filter(None, re.split(r'[^a-zA-Z\']', temp[0].lower()))

        # Then we will move all stop words such as 'to', 'I', 'that', and 'the'
        # And apply the Wordnet Lemmatizer to remove any affix
        stop = set(stopwords.words('english'))
        lmtzr = WordNetLemmatizer()
        parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
        entries.append(parsed)
        
        # Split data based on its label and sperate them into different group
        # this make sure the training data is well balanced.
        if label == '1' and positiveCnt < 400:
            train_X.append(parsed)
            train_Y.append(label)
            positiveCnt += 1
            continue
        if label == '0' and negativeCnt < 400:
            train_X.append(parsed)
            train_Y.append(label)
            negativeCnt += 1
            continue
        test_X.append(parsed)
        test_Y.append(label)
        

print 'There are totally ' + str(len(train_X)) + ' reviews for training'
print str(len(test_X)) + ' reviews for testing.'

There are totally 2400 reviews for training
600 reviews for testing.


## d) Bag of Words model
Extract features and then represent each review using bag of words model, i.e.,  
every word in the review becomes its own element in a feature vector.In order to do this,  
1. make one pass through all the reviews in the training set and build a dictionary of unique words.  
2. Then, make another pass through the review in both the training set and testing set and count up the occurrences of each word in your dictionary. 

The ith element of a review’s feature vector is the number of occurrences of the i th dictionary word in the review.  
Implement the bag of words model and report feature vectors of any two reviews in the training set.

In [14]:
"""
Create a table with all words
    the table is a 1*d table, with each parsed word

If directly print the matrix, there will be char u'*', it is because json parser read string
    as unicode. It will be gone when fetch directly (e.g. ingredientsTable[0] print will not show the u')
"""
def BagOfWordModel(train_X, test_Y):
    wordTable = list()  # A table contains all words in training set
    for i in range(len(train_X)):
        for j in range(len(train_X[i])):
            if train_X[i][j] not in wordTable:
                wordTable.append(train_X[i][j])
    # wordTable = list(wordTable)

    train_X_data = np.zeros((len(train_X), len(wordTable)))  # A table with 0 and 1 to indicate
    for i in range(len(train_X)):
        for j in range(len(train_X[i])):
            train_X_data[i][wordTable.index(train_X[i][j])] += 1
        
    test_X_data = np.zeros((len(test_X), len(wordTable)))  # A table with 0 and 1 to indicate
    for i in range(len(test_X)):
        for j in range(len(test_X[i])):
            if test_X[i][j] in wordTable:  # Only add words that included in the train set
                test_X_data[i][wordTable.index(test_X[i][j])] += 1
                
    print train_X_data[0]
    print train_X_data[1]
    print train_X_data[0].shape
    return train_X_data, test_X_data, wordTable

## e) Pick your postprocessing strategy

In [15]:
# here i choose to use L1 Norm postprocessing strategy
# this part to be discussed. not very sure
def postprocessing(train_X_data):
    for i in range(len(train_X_data)):
        x = sum([k*k for k in train_X_data[i]])
        for j in range(len(wordTable)):
            train_X_data[i][j] /= math.sqrt(x)
    return train_X_data

## f) Clustering

In [16]:
# Euclidean distance
def distance(vecA, vecB):
    return sqrt(sum(power(vecA-vecB, 2)))


# use random value to intialize the centroids
def initialCentroids(dataSet, k):
    n = dataSet.shape[1]
    centroids = mat(zeros((k,n)))
    for j in range(n):
        minE = min(dataSet[:,j])
        maxE = max(dataSet[:,j])
        for i in range(k):
            centroids[i,j] = minE + float(maxE - minE) * random.rand()
    return centroids


def kMeans(dataSet, k):
    m = dataSet.shape[0]
    result = mat(zeros((m,2)))
    centroids = initialCentroids(dataSet, k)
    converged = False
    while not converged:
        converged = True
        for i in range(m):
            minDist, minIndex = np.inf, -1
            for j in range(k):
                dis = distance(centroids[j,:], dataSet[i,:])
                if dis < minDist:
                    minDist, minIndex = dis, j
            if result[i,0] != minIndex: converged = False
            result[i,:] = minIndex, minDist*minDist
        for ci in range(k):
            points = []
            for i in range(m):
                if result[i,0] == ci: points.append(dataSet[i])
            centroids[ci,:] = mean(array(points), axis = 0)
    return centroids, result

In [17]:
# Cluster Evaluator
# compute the ratio of positive & negative previews in each cluster
import matplotlib.pyplot as plt
def evaluate_cluster(train_X_data):
    centroids, result = kMeans(train_X_data, 2)
    #colors=['#000000','#FFFFFF','#FF0000','#00FF00','#0000FF']
    print centroids
    #for i in range(2):
    #    plt.scatter(result[labels==i,0], result[labels==i,1], c=colors[i])
    #    plt.scatter(centroids[i,0], centroids[i,1], c=colors[i], marker='x', s=200, linewidths=2)
    #plt.show()
    
    part1 = []
    part2 = []
    for a,b,c in zip(train_X_data, result.tolist(),train_Y):
        if b[0] == 0: part1.append((a,c))
        else: part2.append((a,c))
    cnt1, cnt2 = 0, 0
    for m,n in part1:
        if n == '0': cnt1 += 1
        else: cnt2 += 1
    cnt3, cnt4 = 0,0
    for m,n in part2:
        if n == '0': cnt3 += 1
        else: cnt4 += 1
    print "cluster1 has " + str(len(part1)) + " reviews"
    print "negative: " + str(cnt1) +" ",
    print "posive: " + str(cnt2)
    print "cluster2 has " + str(len(part2)) + " reviews"
    print "negative: " + str(cnt3) +" ",
    print "posive: " + str(cnt4)

## h) N-gram Model. N = 2

In [18]:
def NgramModel(train_X, test_X):
    dictionary = list()
    for i in range(len(train_X)):
        for j in range(len(train_X[i])-1):
            pair = train_X[i][j]+" "+train_X[i][j+1]
            if pair not in dictionary:
                dictionary.append(pair)
    #dictionary = list(dictionary)
    train_X_data = np.zeros((len(train_X), len(dictionary)))  # A table with 0 and 1 to indicate
    for i in range(len(train_X)):
        for j in range(len(train_X[i])-1):
            pair = train_X[i][j]+" "+train_X[i][j+1]
            train_X_data[i][dictionary.index(pair)] += 1
        
    test_X_data = np.zeros((len(test_X), len(dictionary)))  # A table with 0 and 1 to indicate
    for i in range(len(test_X)):
        for j in range(len(test_X[i])-1):
            pair = test_X[i][j]+" "+test_X[i][j+1]
            if pair in dictionary:  # Only add words that included in the train set
                test_X_data[i][dictionary.index(pair)] += 1
    return train_X_data, test_X_data, dictionary

In [19]:
import heapq
from sklearn.metrics import confusion_matrix

print "Clustering on Bag of Word Model"
print
train, test, wordTable= BagOfWordModel(train_X, test_X)
# train = postprocessing(train)
evaluate_cluster(train)
print
print "Logistic Regression with Bag of Word Model"
cls = linear_model.LogisticRegression()
print "accuracy:",
res = cls.fit(train, train_Y)
print res.score(test, test_Y)
print confusion_matrix(test_Y, res.predict(test))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    print wordTable[np.where(res.coef_[0] == i)[0]]


print
print "Clustering on N-gram Model"
print
train, test, wordTable = NgramModel(train_X, test_X)
# train = postprocessing(train)
evaluate_cluster(train)
print
print "Logic Regression with N-gram Model"
cls = linear_model.LogisticRegression()
print "accuracy:",
res = cls.fit(train, train_Y)
print res.score(test, test_Y)
print confusion_matrix(test_Y, res.predict(test))
nmax = heapq.nlargest(10, res.coef_[0])
for i in nmax:
    print wordTable[np.where(res.coef_[0] == i)[0]]

Clustering on Bag of Word Model

[ 1.  1.  1. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
(4091,)
[[ 0.01791667  0.00541667  0.00833333 ...,  0.00041667  0.00041667
   0.00041667]
 [        nan         nan         nan ...,         nan         nan
          nan]]
cluster1 has 2400 reviews
negative: 1200  posive: 1200
cluster2 has 0 reviews
negative: 0  posive: 0

Logistic Regression with Bag of Word Model
accuracy: 0.803333333333
[[257  43]
 [ 75 225]]
great
delicious
love
excellent
nice
loved
fantastic
awesome
amazing
liked

Clustering on N-gram Model



/Users/Zephyr/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


[[        nan         nan         nan ...,         nan         nan
          nan]
 [ 0.00041667  0.00041667  0.00041667 ...,  0.00041667  0.00041667
   0.00041667]]
cluster1 has 0 reviews
negative: 0  posive: 0
cluster2 has 2400 reviews
negative: 1200  posive: 1200

Logic Regression with N-gram Model
accuracy: 0.638333333333
[[267  33]
 [184 116]]
work great
highly recommend
one best
great phone
great product
food good
really good
easy use
good price
great food


/Users/Zephyr/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


## i) PCA for bag of word model.
The feature in *bag of words* has large redundancy.  
Implement PCA first to reduce dimension of features to 10, 50, 100.

In [20]:
def PCA(train, test, r):
    # subtract mean
    mean_vector = np.mean(train, axis=0)
    X = train - mean_vector
    X = np.dot(X.transpose(), X)
    U, s, V = np.linalg.svd(X, full_matrices = True)
    F = np.dot(train, (V[:r,:]).T)
    K = np.dot(test, (V[:r,:]).T)
    return F, K
# test PCA implement choose r = 10, 50, 100
train, test, wordTable = BagOfWordModel(train_X, test_X)
train, test = PCA(train, test, 100)
cls = linear_model.LogisticRegression()
print "accuracy with r = 100:",
print cls.fit(train, train_Y).score(test, test_Y)
train, test = PCA(train, test, 50)
cls = linear_model.LogisticRegression()
print "accuracy with r = 50:",
print cls.fit(train, train_Y).score(test, test_Y)
train, test = PCA(train, test, 10)
cls = linear_model.LogisticRegression()
print "accuracy with r = 10",
print cls.fit(train, train_Y).score(test, test_Y)

[ 1.  1.  1. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
(4091,)
accuracy with r = 100: 0.696666666667
accuracy with r = 50: 0.681666666667
accuracy with r = 10 0.613333333333
